In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict, Audio, concatenate_datasets
import os

# List of dataset directories
data_dirs = [
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca",
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca2",
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca3",
]

# Function to load a single dataset
def load_local_dataset(data_dir):
    path = os.path.join(data_dir, "validated.tsv")
    validated_data = pd.read_csv(path, sep='\t')

    # Keep only necessary columns, such as "path" and "sentence"
    validated_data = validated_data[["path", "sentence"]]
    validated_data["path"] = validated_data["path"].apply(lambda x: os.path.join(data_dir, "clips", x))

    # Convert to Hugging Face Dataset
    dataset = Dataset.from_pandas(validated_data)

    # Ensure audio files are loaded properly
    dataset = dataset.cast_column("path", Audio(sampling_rate=16_000))
    return dataset

# Load all datasets and concatenate them
datasets = [load_local_dataset(data_dir) for data_dir in data_dirs]
dataset = concatenate_datasets(datasets)

# Print the result
print(dataset)


Dataset({
    features: ['path', 'sentence'],
    num_rows: 10906
})


d:\Users\Usuario\anaconda3\Lib\site-packages\datasets\table.py:1381: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
d:\Users\Usuario\anaconda3\Lib\site-packages\datasets\table.py:1407: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [3]:
import pandas as pd
import json
import os

# List of dataset directories
data_dirs = [
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca",
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca2",
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca3",
    # Add more directories as needed
]

# Initialize an empty set for the vocabulary
vocab = set()

# Iterate through each dataset directory
for data_dir in data_dirs:
    # Load the validated.tsv file
    validated_path = os.path.join(data_dir, "validated.tsv")
    if os.path.exists(validated_path):
        validated_data = pd.read_csv(validated_path, sep='\t')

        # Extract unique characters from sentences
        for sentence in validated_data["sentence"]:
            vocab.update(list(sentence))  # Add all characters in the sentence to the vocab set

# Convert the set to a sorted list for consistency
vocab = sorted(vocab)

# Add special tokens
vocab_dict = {char: idx for idx, char in enumerate(vocab)}
vocab_dict["[UNK]"] = len(vocab_dict)       # Unknown token
vocab_dict["[PAD]"] = len(vocab_dict)       # Padding token
vocab_dict["|"] = len(vocab_dict)           # Word delimiter (used as a separator for CTC decoding)

# Save vocabulary to a JSON file
output_path = "vocab3.json"
with open(output_path, "w", encoding="utf-8") as vocab_file:
    json.dump(vocab_dict, vocab_file)

print(f"Vocabulary saved to {output_path}")


Vocabulary saved to vocab3.json
